In [4]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
print("✅ 한글 폰트 설정 완료!") 

✅ 한글 폰트 설정 완료!


In [9]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

# 1) Bell 상태 회로 생성
qc = QuantumCircuit(2,2)
qc.h(0)
qc.cx(0,1)
qc.measure([0,1],[0,1])

print(qc.draw())
print()

# 2) 로컬 시뮬레이터 테스트
simulator = AerSimulator()
sim_job = simulator.run(qc, shots=1000)
sim_result = sim_job.result()
sim_counts = sim_result.get_counts()

print(" 시뮬 결과:", sim_counts)
print()

# 3) IBM 양자컴퓨터

QiskitRuntimeService.save_account(
    channel='ibm_quantum_platform',
    token = '',
    overwrite = True)

service = QiskitRuntimeService()

backend = service.least_busy(operational=True, simulator=False)
print(f" 사용할 장비 : {backend.name}")

transpiled_qc = transpile(qc, backend=backend, optimization_level=3)

sampler = SamplerV2(backend)
job = sampler.run([transpiled_qc], shots=1024)

real_result = job.result()
# 결과
print(" 사용 가능한 속성 : ", dir(real_result[0].data))

try:
    data_keys = list(real_result[0].data.__dic__.keys())
    print(f"데이터 키: {data_keys}")

    measurement_key = data_keys[0] if data_keys else 'c'
    real_counts = getattr(real_result[0].data, measurement_key).get_counts()

except Exception as e:
    print(f" 오류: {e}")

    try:
        real_counts = real_result[0].data.c.get_counts()
    except:
        from collections import Counter
        bitstrings = [fromat(x, '02b') for x in real_result[0].data.c]
        real_counts = dict(Counter(bitstrings))

print("\n" + "="*60)
print("=== 결과 비교 ===")
print(f"🖥️  시뮬레이터 (이상적): {sim_counts}")
print(f"🔬 실제 양자컴 (노이즈): {real_counts}")
print("="*60)

total_shots = sum(real_counts.values())
ideal_shots = real_counts.get('00', 0) + real_counts.get('11', 0)
fidelity = ideal_shots / total_shots*100

print(f"\n📊 분석:")
print(f"   전체 샷 수: {total_shots}")
print(f"   이상적 결과('00'+'11'): {ideal_shots} ({fidelity:.1f}%)")
print(f"   오류 비트('01'+'10'): {total_shots - ideal_shots} ({100-fidelity:.1f}%)")
print(f"\n💡 Bell 상태 충실도: {fidelity:.2f}%")
print("   → 85% 이상이면 성공! 🎉")

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 

 시뮬 결과: {'00': 485, '11': 515}



qiskit_runtime_service.__init__:WARNING:2025-12-17 20:49:29,307: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: dotnanimoe. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-17 20:49:29,970: Loading instance: dotnanimoe, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-17 20:49:33,303: Using instance: dotnanimoe, plan: open


 사용할 장비 : ibm_torino
 사용 가능한 속성 :  ['_FIELDS', '_FIELD_TYPES', '_RESTRICTED_NAMES', '_SHAPE', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__non_callable_proto_members__', '__parameters__', '__protocol_attrs__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_data', '_is_protocol', '_is_runtime_protocol', '_shape', 'c', 'items', 'keys', 'ndim', 'shape', 'size', 'values']
 오류: 'DataBin' object has no attribute '__dic__'

=== 결과 비교 ===
🖥️  시뮬레이터 (이상적): {'00': 485, '11': 515}
🔬 실제 양자컴 (노이즈): {'11': 504, '00': 499, '01': 4, '10': 17}

📊 분석:


In [10]:
# 결과가 나온 후 실행
real_result = job.result()

print("=== 결과 구조 분석 ===")
print("1. 결과 타입:", type(real_result))
print("2. 첫 번째 결과:", type(real_result[0]))
print("3. 데이터 타입:", type(real_result[0].data))
print("4. 사용 가능한 속성:", dir(real_result[0].data))
print("5. 실제 데이터:")

# 모든 속성 출력
for attr in dir(real_result[0].data):
    if not attr.startswith('_'):
        try:
            value = getattr(real_result[0].data, attr)
            print(f"   - {attr}: {type(value)}")
        except:
            pass

=== 결과 구조 분석 ===
1. 결과 타입: <class 'qiskit.primitives.containers.primitive_result.PrimitiveResult'>
2. 첫 번째 결과: <class 'qiskit.primitives.containers.sampler_pub_result.SamplerPubResult'>
3. 데이터 타입: <class 'qiskit.primitives.containers.data_bin.DataBin'>
4. 사용 가능한 속성: ['_FIELDS', '_FIELD_TYPES', '_RESTRICTED_NAMES', '_SHAPE', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__non_callable_proto_members__', '__parameters__', '__protocol_attrs__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_data', '_is_protocol', '_is_runtime_protocol', '_sh

In [1]:
#충실도 개선 실험
for opt_level in [0,1,2,3]:
    transpiled = transpile(qc, backend, optimization_level=opt_level)
    print(f"level {opt_level}: {transpiled.size()} gates, depth {transpiled.depth()}")

NameError: name 'transpile' is not defined

In [2]:
# 백엔드 성능 정보
props = backend.properties()
print(f"큐비트 0 T1: {props.t1(0) * 1e6:.1f} μs")  # 이완 시간
print(f"큐비트 0 T2: {props.t2(0) * 1e6:.1f} μs")  # 결맞음 시간
print(f"CNOT 오류율: {props.gate_error('cx', [0, 1]) * 100:.2f}%")

NameError: name 'backend' is not defined